# Heroes of Pymoli

Objective: Perform analyses on game purchase data to gain meainful insights.

### Load data into pandas dataframe

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
df_orig = pd.read_csv(file_to_load)
df = df_orig.copy()
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


### Player Count

* Total number of unique players


In [2]:
# Number of unique players
player_count = df['SN'].nunique(dropna=False)
print(f'Number of unique players: {player_count}')

Number of unique players: 576


### Purchasing Analysis (Total)

* Number of Unique Items
* Average Purchase Price
* Total Number of Purchases
* Total Revenue

In [3]:
# Calculate each of the metrics shown above and store the values in a list
totals = [df['Item ID'].nunique(dropna=False), f"${df['Price'].mean():.2f}", len(df.index), f"${df['Price'].sum():,.2f}"]

# Create list of labels
labels = ['Number of unique items', 'Average price of all purchases', 'Number of purchases', 'Total revenue']

# Display a temporary DataFrame showing results
pd.DataFrame([totals], columns=labels)

,Number of unique items,Average price of all purchases,Number of purchases,Total revenue
0,179,$3.05,780,"$2,379.77"


### Gender Demographics

* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

In [4]:
# Create new DataFrame with unique SN and Gender
df_players = df[['SN', 'Gender']].copy().drop_duplicates()

# Create new DataFrame indexed by Gender with player count by Gender
df_gender = pd.DataFrame(df_players['Gender'].copy().value_counts())

# Rename column
df_gender.columns = ['Count of players']

# Add column with percent of players by Gender
df_gender['Percent of players'] = [f"{x / df_gender['Count of players'].sum():.2%}" for x in df_gender['Count of players']]

# Display DataFrame
df_gender

,Count of players,Percent of players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


### Purchasing Analysis (Gender)

Calculate each by gender:

* Purchase Count
* Average Purchase Price
* Total Purchase Value
* Average Purchase Total per Person by Gender

In [5]:
# Create GroupBy object on Gender
gb_gender = df.groupby(['Gender'])

# Create DataFrame grouped by Gender and SN, with sum of purchases
gb_gen_sn = df.groupby(['Gender', 'SN'])['Price'].sum()

# Use previous DataFrames to construct new DataFrame with analysis items and assign temporary column names
df_gend_purch = pd.DataFrame({'PC': gb_gender['SN'].count(),
             'APP': gb_gender['Price'].mean(),
             'TPV': gb_gender['Price'].sum(),
             'APTPP': gb_gen_sn.groupby(['Gender']).mean()})

# Reformat monetary values as currency
df_gend_purch['APP'] = df_gend_purch['APP'].map("${:,.2f}".format)
df_gend_purch['TPV'] = df_gend_purch['TPV'].map("${:,.2f}".format)
df_gend_purch['APTPP'] = df_gend_purch['APTPP'].map("${:,.2f}".format)

# Rename columns
df_gend_purch.columns = ['Purchase count', 'Average purchase price', 'Total purchase value', 'Avg purchase total per person']

# Display DataFrame
df_gend_purch

,Purchase count,Average purchase price,Total purchase value,Avg purchase total per person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


### Age Demographics

Calculate age demographics for players using bins of 5 years (i.e. &lt;10, 10-14, 15-19, etc.):

* Count of players
* Percentage of players

In [6]:
# Make copy of working DataFrame
df_age_group = df[['Age', 'SN', 'Price']].copy()

# Makes lists for bins and labels
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
labels = ['<10', '10-14', '15-19', '20-24', '25-29','30-34','35-39','40+']

# Create column with bin labels
df_age_group["Age Range"] = pd.cut(df_age_group["Age"], bins, labels=labels)

# Drop records with duplicate SN (due to players making multiple purchases) leaving unique SN
df_players = df_age_group[['SN','Age Range']].drop_duplicates().reset_index(drop=True)

# Create dataframe indexed by bin label with player count
df_age_count = df_players['Age Range'].value_counts(sort=False).to_frame(name='Player Count')

# Rename index column
df_age_count.index.name = 'Age Range'

# Calculate new column with percentages of player counts
df_age_count['Percent of Players'] = [f"{x / player_count:.2%}" for x in df_age_count['Player Count']]

# Display DataFrame
df_age_count

,Player Count,Percent of Players
Age Range,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


### Purchasing Analysis (Age)

Using the same age group bins, calculate the following statistics using purchase data:

* Purchase Count
* Average Purchase Price
* Total Purchase Value
* Average Purchase Total per Person by Age Group

In [7]:
# Using DataFrame created previously, calculate series for first three analysis items
s_count = df_age_group.groupby(['Age Range'])['Price'].count()
s_mean = df_age_group.groupby(['Age Range'])['Price'].mean()
s_sum = df_age_group.groupby(['Age Range'])['Price'].sum()

# Calculate series of average purchase total per person
s_player_totpurch = df_age_group.groupby(['SN','Age Range'], observed=True)['Price'].sum()
s_meantot = s_player_totpurch.groupby(['Age Range']).mean()

# Combine series into DataFrame
df_age_purch = pd.concat([s_count, s_mean, s_sum, s_meantot], axis=1)

# Reformat monetary values as currency
for ind in range(1,4):
    df_age_purch.iloc[:, ind] = df_age_purch.iloc[:, ind].map("${:,.2f}".format)

# Rename columns
df_age_purch.columns = ['Purchase count', 'Average purchase price', 'Total purchase value', 'Avg purchase total per person']

# Display DataFrame
df_age_purch

,Purchase count,Average purchase price,Total purchase value,Avg purchase total per person
Age Range,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


### Top Spenders

Identify the the top 5 spenders in the game by total purchase value, then tabulate the following:

* SN
* Purchase Count
* Average Purchase Price
* Sum of Purchases

In [8]:
# Create first three columns of DataFrame by concatenating two GroupBy series
df_purch = pd.concat([df.groupby(['SN'])['Item ID'].count(), df.groupby(['SN'])['Price'].sum()], axis=1)

# Rename columns
df_purch.columns = ['Purchase count', 'Sum of purchases']

# Sort descending on 'Sum of purchases'
df_purch.sort_values(['Sum of purchases'],  ascending=False, inplace=True)

# Calculate average purchase price and insert at column index 1
df_purch.insert(1, 'Average purchase price', round(df_purch['Sum of purchases'] / df_purch['Purchase count'], 2))

# Reformat monetary values as currency
for ind in range(1,3):
    df_purch.iloc[:, ind] = df_purch.iloc[:, ind].map("${:,.2f}".format)

# Display DataFrame
df_purch.head(5)

,Purchase count,Average purchase price,Sum of purchases
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


### Most Popular Items

Identify the most popular items first by purchase count and second by total purchase value. 
List at least the first five items, but add more items to the list as needed if there are 
ties. Tabulate the following for the most popular items:

* Item ID
* Item Name
* Purchase Count
* Item Average Price
* Total Purchase Value

In [10]:
#######################
### IMPORTANT NOTES ###
#######################

# 1. The price of some items in the sample data is not fixed, e.g. Item ID 132 is $3.19 
#    through 'Purchase ID' 603, then increases to $3.33 thereafter. I named the 4th  
#    bullet "Item Average Price" to reflect this.
#
# 2. There are 4 items with a purchase count of 9 tied for 3rd. As a result, one item would 
#    not be displayed if only 5 items were shown in the summary table. To reflect this I modified the
#    I modified the instructions to account for ties and show more items as needed. I also did a 
#    secondary descending sort on 'Total purchase value'

# Create copy of working DataFrame
df_item = df[['Item ID', 'Item Name', 'Price']]

# Create series indexed by GroupBy on 'Item ID' and 'Item Name' for first three analysis items
s_item_count = df.groupby(['Item ID', 'Item Name'])['Item ID'].count()
s_item_price = df.groupby(['Item ID', 'Item Name'])['Price'].mean()
s_item_tot = df.groupby(['Item ID', 'Item Name'])['Price'].sum()

# Concatenate the series into a DataFrame
df_item_table = pd.concat([s_item_count, s_item_price, s_item_tot], axis=1)

# Rename columns
df_item_table.columns = ['Purchase count', 'Item average price', 'Total purchase value']

# Make a copy of the DataFrame to allow 'df_item_table' to be reused for subsequent analysis
df_item_pop = df_item_table.copy()

# Sort DataFrame descending
df_item_pop.sort_values(['Purchase count', 'Total purchase value'],  ascending=False, inplace=True)

# Reformat monetary values as currency
for ind in range(1,3):
    df_item_pop.iloc[:, ind] = df_item_pop.iloc[:, ind].map("${:,.2f}".format)
    
# Display 6 items so all items with "Purchase count" of 9 are shown
df_item_pop.head(6)

,,Purchase count,Item average price,Total purchase value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
132,Persuasion,9,$3.22,$28.99


### Most Profitable Items

Identify the 5 most profitable items by total purchase value, then tabulate the following:

* Item ID
* Item Name
* Purchase Count
* Item Average Price
* Total Purchase Value

In [11]:
# Make a copy of 'df_item_table' which was an intermediate step in the previous analysis
df_item_prof = df_item_table.copy()

# Sort descending
df_item_prof.sort_values(['Total purchase value'],  ascending=False, inplace=True)

# Reformat monetary values as currency
for ind in range(1,3):
    df_item_prof.iloc[:, ind] = df_item_prof.iloc[:, ind].map("${:,.2f}".format)
    
# Display DataFrame
df_item_prof.head(5)

,,Purchase count,Item average price,Total purchase value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80


## Analysis - Identify Three Observable Trends

1. The majority of players identify as 'Male' at nearly 85%. However, 'Female' players \
spend nearly 10% more than 'Male' players (mean purchase \\$4.47 versus \\$4.07, respectively).
2. Over 75% of the players are in the age range 15 through 29 and contribute a combined revenue \
of 76%.
3. There are 780 purchases distributed among 576 players. The difference between these is 204, \
indicating that a large percentage of purchases reflect multiple purchases from players. Since \
the highest one-player purchase count is 5, this suggest that a large number of players are making \
multiple purchases (as opposed to a few players dominating multiple purchases).